In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import cv2

In [0]:
import pickle

with open('drive/My Drive/Machine_Learning-prj/face_detection/X_train_triplet.pkl', 'rb') as f:
    X = pickle.load(f)
    X = np.array(X)
#     X = np.expand_dims(X, axis=3)
    X = X/255.
with open('drive/My Drive/Machine_Learning-prj/face_detection/y_train_triplet.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [4]:
np.shape(X), np.shape(y)

((8405, 221, 221, 3), (8405,))

In [0]:
# import sklearn.preprocessing

# label_binarizer = sklearn.preprocessing.LabelBinarizer()

# label_binarizer.fit(range(4))

# y = label_binarizer.transform(y)
# y_val = label_binarizer.transform(y_val)

# print(y.shape)

In [6]:
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from keras import applications

Using TensorFlow backend.


In [0]:
import tensorflow as tf
from keras import backend as K
import h5py


def convnet_model_():
    vgg_model = applications.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda x_: K.l2_normalize(x,axis=1))(x)
#     x = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(221, 221, 3))
    first_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
#     first_max = Lambda(K.l2_normalize)(first_max)
    first_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(first_max)

    second_input = Input(shape=(221, 221, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)
#     second_max = Lambda(K.l2_normalize)(second_max)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
#     l2_norm_final = Lambda(K.l2_normalize)(emb)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model

In [8]:
deep_rank_model = deep_rank_model()
deep_rank_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 221, 221, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 221, 221, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 221, 221, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 110, 110, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [0]:
batch_size = 24

_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0 - _EPSILON)
    loss = 0.
    g = 1.
    for i in range(0, batch_size, 3):
        try:
            q_embedding = y_pred[i]
            p_embedding = y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p = K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = loss + g + D_q_p - D_q_n
        except:
            continue
    loss = loss/batch_size*3
    return K.maximum(loss, 0)

In [0]:
deep_rank_model.compile(loss=_loss_tensor, optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True))

In [0]:
def image_batch_generator(images, labels, batch_size):
    labels = np.array(labels)
    while True:
        batch_paths = np.random.choice(a = len(images), size = batch_size//3)
        input_1 = []
        
        for i in batch_paths:
            pos = np.where(labels == labels[i])[0]
            neg = np.where(labels != labels[i])[0]
            
            j = np.random.choice(pos)
            while j == i:
                j = np.random.choice(pos)
             
            k = np.random.choice(neg)
            while k == i:
                k = np.random.choice(neg)
            
            input_1.append(images[i])
            input_1.append(images[j])
            input_1.append(images[k])

        input_1 = np.array(input_1)
        input = [input_1, input_1, input_1]
        yield(input, np.zeros((batch_size, )))

In [12]:
from keras.models import load_model

print("Loading pre-trained weight")
deep_rank_model.load_weights('drive/My Drive/Machine_Learning-prj/face_detection/triplet_weight.hdf5')

Loading pre-trained weight


In [0]:
from keras.callbacks import ModelCheckpoint

filepath = 'drive/My Drive/Machine_Learning-prj/face_detection/triplet_weight.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [14]:
deep_rank_model.fit_generator(generator=image_batch_generator(X, y, batch_size),
                   steps_per_epoch=len(X)//batch_size,
                   epochs=2000,
                   verbose=1,
                   callbacks=callbacks_list)
     

Epoch 1/2000
350/350 [==============================] - 283s 810ms/step - loss: 0.4185

Epoch 00001: loss improved from inf to 0.41851, saving model to drive/My Drive/Machine_Learning-prj/face_detection/triplet_weight.hdf5
Epoch 2/2000
350/350 [==============================] - 272s 778ms/step - loss: 0.4180

Epoch 00002: loss improved from 0.41851 to 0.41805, saving model to drive/My Drive/Machine_Learning-prj/face_detection/triplet_weight.hdf5
Epoch 3/2000
350/350 [==============================] - 272s 779ms/step - loss: 0.4041

Epoch 00003: loss improved from 0.41805 to 0.40406, saving model to drive/My Drive/Machine_Learning-prj/face_detection/triplet_weight.hdf5
Epoch 4/2000
350/350 [==============================] - 272s 778ms/step - loss: 0.4037

Epoch 00004: loss improved from 0.40406 to 0.40370, saving model to drive/My Drive/Machine_Learning-prj/face_detection/triplet_weight.hdf5
Epoch 5/2000
350/350 [==============================] - 272s 778ms/step - loss: 0.4259

Epoch 00

KeyboardInterrupt: ignored